In [1]:
import pandas as pd

daily_intake = {'Calories':{'minimum':1500,'maximum':2500},
                'Cholesterol_mg':{'minimum':30,'maximum':240},
                'Total_Fat_g':{'minimum':20,'maximum':70}, 
                'Sodium_mg':{'minimum':800,'maximum':2000},
                'Carbohydrates_g':{'minimum':130,'maximum':450}, 
                'Dietary_Fiber_g':{'minimum':125,'maximum':250},
                'Protein_g':{'minimum':60,'maximum':100},
                'Vit_A_IU':{'minimum':1000,'maximum':10000},
                'Vit_C_IU':{'minimum':400,'maximum':5000},
                'Calcium_mg':{'minimum':700,'maximum':1500},
                'Iron_mg':{'minimum':10,'maximum':40}
               }
daily_intake

{'Calcium_mg': {'maximum': 1500, 'minimum': 700},
 'Calories': {'maximum': 2500, 'minimum': 1500},
 'Carbohydrates_g': {'maximum': 450, 'minimum': 130},
 'Cholesterol_mg': {'maximum': 240, 'minimum': 30},
 'Dietary_Fiber_g': {'maximum': 250, 'minimum': 125},
 'Iron_mg': {'maximum': 40, 'minimum': 10},
 'Protein_g': {'maximum': 100, 'minimum': 60},
 'Sodium_mg': {'maximum': 2000, 'minimum': 800},
 'Total_Fat_g': {'maximum': 70, 'minimum': 20},
 'Vit_A_IU': {'maximum': 10000, 'minimum': 1000},
 'Vit_C_IU': {'maximum': 5000, 'minimum': 400}}

In [2]:
diet = pd.read_csv('diet.csv')
diet.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,$0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",$0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",$0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,$0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",$0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


In [3]:
diet.tail()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
62,"Crm Mshrm Soup,W/Mlk",$0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn Soup,W/Watr",$0.67,1 C (8 Fl Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
diet.dropna(inplace = True)


In [5]:
diet['Foods'] = [x.replace(' ','_').replace(',','').replace('/','').replace('-','_') for x in diet['Foods']]

In [6]:
diet['Price/ Serving'] = [x.replace('$','') for x in diet['Price/ Serving']]

In [7]:
diet['Price/ Serving']= diet['Price/ Serving'].astype('float',inplace = True)

In [8]:
from pulp import *

In [9]:
problem = LpProblem('Diet Problem', LpMinimize)


#Create the variables
variables = {}
for row in diet.iterrows():
    variable = 'x_{}'.format(str(row[0]))
    name = row[1][0]
    variables.update({variable:LpVariable(name, 0,None)})


In [10]:
#build the objective function and add it to problem
obj_func = 0
for k,v in variables.items():
    name = v.getName()
    price = diet[diet['Foods']==name]['Price/ Serving'].values[0]
    obj_func += v*price
    
problem += obj_func, "Total Cost of Ingredients"

In [11]:
#build constraints 
from collections import defaultdict
const_dict = defaultdict(float)
#the percentage constraint
perc = 0
for k,v in variables.items():
    name = v.getName()
    row = diet[diet['Foods']==name].loc[:,'Calories':]
    for i,vals in row.items():
        const_dict[i]+= v*vals.values[0]
    perc += v

In [12]:
#add constraints to model with minimums and maximums
for key,value in const_dict.items():
    k = key.replace(' ','_').replace(',','').replace('/','').replace('-','_')
    dly_intk_min = daily_intake[k]['minimum']
    dly_intk_max = daily_intake[k]['maximum']
    problem+= value >= dly_intk_min, '{}_min_requirement'.format(k)
    problem+= value <= dly_intk_max, '{}_max_requirement'.format(k)
    

In [13]:
# adding the percentage constraint
problem += perc == 100, "PercentagesSum"

In [16]:
problem.writeLP("DietModel.lp")
problem.solve()
print("Status:", LpStatus[problem.status])
for v in problem.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue, 'servings')

Status: Optimal
Celery_Raw = 52.374743 servings
Frozen_Broccoli = 0.46362345 servings
KiwifruitRawFresh = 20.291596 servings
LettuceIcebergRaw = 20.287523 servings
Poached_Eggs = 0.14184397 servings
PopcornAir_Popped = 5.7019086 servings
Tofu = 0.73876257 servings
